In [288]:
import numpy as np
from tqdm import tqdm
import datetime 
import zarr
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, poisson
from scipy.special import softmax
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
from matplotlib import animation
from IPython.display import HTML

def generate(num_predators, num_venomous_prey, num_mimics, d=2):

    predators_detectors_initial_mean = np.array([1 / np.sqrt(2) for _ in range(d)])
    predators_detectors_initial_cov = 0.4**2 * np.eye(d)
    
    venomous_signals_initial_mean = np.array([1 / np.sqrt(2), *[-1 / np.sqrt(2) for _ in range(d-1)]])
    venomous_signals_initial_cov = 0.4**2 * np.eye(d)
    
    mimic_signals_initial_mean = np.array([*[-1 / np.sqrt(2) for _ in range(d-1)], 1 / np.sqrt(2)])
    mimic_signals_initial_cov = 0.4**2 * np.eye(d)
    
    detectors =         np.random.multivariate_normal(mean=predators_detectors_initial_mean,
                                                      cov=predators_detectors_initial_cov,
                                                      size=num_predators)
    venomous_signals =  np.random.multivariate_normal(mean=venomous_signals_initial_mean,
                                                      cov=venomous_signals_initial_cov,
                                                      size=num_venomous_prey)
    mimic_signals =     np.random.multivariate_normal(mean=mimic_signals_initial_mean,
                                                      cov=mimic_signals_initial_cov,
                                                      size=num_mimics)
    signals = np.vstack((venomous_signals, mimic_signals))
    
    risk_tols = np.random.exponential(scale=1., size=num_predators)
    venom_levels = np.concatenate((np.random.uniform(0., 1., size=num_venomous_prey), np.zeros(num_mimics)))

    return detectors, signals, risk_tols, venom_levels

def similarity(detectors, signals, phenotype_type='vector'):
    """
    Calculate the similarities between lists of signals and detectors. Signals and detectors can either all be vectors
    or bitstrings.
    """
    match phenotype_type:
        case 'vector':
            dist = pairwise_distances(detectors, signals)
            return - dist
            # sim = cosine_similarity(detectors, signals)  # Matrix of cosine similarities
            # p = (sim + 1)/2
            # return np.log(p/(1-p))
        case 'bitstring':
            d = signals.shape[1]
            # Calculate matrix of hamming distances
            detectors_expanded = detectors[:, np.newaxis, :]
            signals_expanded = signals[np.newaxis, :, :]
            hamming_distances = np.sum(detectors_expanded != signals_expanded, axis=2)
            return 1 - hamming_distances/d  # Matrix of fractions of bits that are the same
        case _:
            raise NotImplementedError

def calculate_detection_matrix(detectors, signals, risk_tols, phenotype_type='vector'):
    similarity_matrix = similarity(detectors, signals, phenotype_type=phenotype_type)    
    # map to [0, 1]
    f = lambda x: 1 / (1 + np.exp(-x))
    detection_matrix = f(similarity_matrix)
    return detection_matrix

def calculate_predation_matrix(detectors, signals, risk_tols, handling_time, attack_freq, phenotype_type='vector'):
    detection_matrix = calculate_detection_matrix(detectors, signals, risk_tols, phenotype_type)
    n_prey = len(signals)
    interaction_prob = attack_freq / (1 + attack_freq * handling_time * n_prey)
    return interaction_prob * detection_matrix

def sample_predators(predation_matrix, venom_levels, pred_conversion_ratio):
    """
    Sample the predator population based on the predation matrix and venom levels.
    """
    num_predators, num_prey = predation_matrix.shape
    e = predation_matrix.sum(1) * (1 - venom_levels * predation_matrix).prod(1)
    c = []
    for i in range(num_predators):
        mean = pred_conversion_ratio / num_predators * e[i]
        n_i = int(poisson.rvs(mean, size=1)[0])
        c.extend([i for _ in range(n_i)])
    return np.array(c, dtype=np.int64)

def sample_prey(predation_matrix, prey_conversion_ratio):
    """
    Sample the prey population based on the predation matrix.
    """
    num_predators, num_prey = predation_matrix.shape
    fitnesses = (1 - predation_matrix).prod(0)    
    consumption = prey_conversion_ratio / num_prey
    c = []
    for i in range(num_prey):
        mean = fitnesses[i] * consumption
        n_i = int(poisson.rvs(mean, size=1)[0])
        c.extend([i for _ in range(n_i)])
    return np.array(c, dtype=np.int64) # a list of indices of prey


def phenotype_crossover(phenotypes, parents, phenotype_type='vector'):

    # phenotypes can be signals or detectors
    assert len(parents) % 2 == 0, 'Crossover not implemented yet for odd numbers of parents'
    parent_phenotypes = phenotypes[parents]
    child_phenotypes = np.zeros_like(parent_phenotypes)
    match phenotype_type:
        case 'vector':
            interpolation_values = np.random.rand(parent_phenotypes.shape[0] // 2)[:, np.newaxis]
            child_phenotypes[::2]  = interpolation_values * parent_phenotypes[::2] \
                                    + (1 - interpolation_values) * parent_phenotypes[1::2]
            child_phenotypes[1::2] = (1 - interpolation_values) * parent_phenotypes[::2] \
                                    + interpolation_values * parent_phenotypes[1::2]
            return child_phenotypes
        case 'bitstring':
            raise NotImplementedError
        case _:
            raise NotImplementedError
        

def phenotype_mutate(phenotypes, mutation_rate=0.01, phenotype_type='vector'):
    """
    Apply random mutations to the bit strings or numerical values.
    """
    if len(phenotypes.shape) == 1:
        d = 1
        num_individuals = len(phenotypes)
    else:
        num_individuals, d = phenotypes.shape

    match phenotype_type:
        case 'vector':
            # noise = np.random.normal(scale=mutation_rate, size=(num_individuals, d))  # scale mutation rate with d?
            noise = np.random.multivariate_normal(mean=np.zeros(d), cov=mutation_rate*np.eye(d), size=num_individuals)
            return phenotypes + noise
        case 'bitstring':
            # For bit strings, flip bits with some probability
            raise NotImplementedError
        case _:
            raise NotImplementedError

def impose_periodic_boundary(vectors, boundary=5):
    """
    Imposes periodic boundary conditions on an array of 2D vectors.
    
    Parameters:
    vectors (np.array): Array of shape (n, 2) containing 2D vectors.
    boundary (float): The boundary value for both x and y dimensions. Default is 5.
    
    Returns:
    np.array: Array of shape (n, 2) with periodic boundary conditions applied.
    """
    # Ensure the input is a numpy array
    vectors = np.array(vectors)
    
    # Apply periodic boundary conditions
    vectors = np.mod(vectors + boundary, 2 * boundary) - boundary
    
    return vectors


def update(detectors, signals, risk_tols, venom_levels, num_venomous, 
           handling_time=1, attack_freq=2, predator_conversion_ratio=1000, prey_conversion_ratio=10000,
           mutation_rate=0.001, phenotype_type='vector'):

    assert np.all(venom_levels[num_venomous:] == 0), 'A mimic has a non-zero venom level'

    predation_matrix = calculate_predation_matrix(detectors, signals, risk_tols, handling_time, attack_freq, phenotype_type=phenotype_type)

    predator_children = sample_predators(predation_matrix, venom_levels, predator_conversion_ratio)
    prey_children = sample_prey(predation_matrix, prey_conversion_ratio)
    
    predator_children_detectors = detectors[predator_children]
    prey_children_signals = signals[prey_children]
    
    predator_childrens_detectors = phenotype_mutate(predator_children_detectors, mutation_rate=mutation_rate, phenotype_type=phenotype_type)
    prey_childrens_signals = phenotype_mutate(prey_children_signals, mutation_rate=mutation_rate, phenotype_type=phenotype_type)

    predator_children_detectors = impose_periodic_boundary(predator_childrens_detectors)
    prey_childrens_signals = impose_periodic_boundary(prey_childrens_signals)
    
    predator_childrens_risk_tols = phenotype_mutate(risk_tols[predator_children], mutation_rate=mutation_rate, phenotype_type=phenotype_type)
    prey_childrens_venoms = venom_levels[prey_children]

    new_num_venomous = (prey_childrens_venoms > 0).sum()

    return predator_childrens_detectors, prey_childrens_signals, predator_childrens_risk_tols, prey_childrens_venoms, new_num_venomous

In [289]:
# Initialize population parameters
num_predators = 100
num_venomous_prey = 200
num_mimics = 200
d = 2
mutation_rate = 0.01

detectors, signals, risk_tols, venom_levels = generate(num_predators, num_venomous_prey, num_mimics, d)

In [294]:
num_generations = 100
ht = 0.5
af = 5
mutation_rate = 0.01
predator_conversion_ratio = 1000
prey_conversion_ratio = 1000

detectors_history    = []
signals_history      = []
risk_tols_history    = []
venom_levels_history = []
nv_history = []

detectors_history.append(detectors)
signals_history.append(signals)
risk_tols_history.append(risk_tols)
venom_levels_history.append(venom_levels)
nv_history.append(num_venomous_prey)

for t in range(num_generations-1):
    print(f"\n Gen {t}")
    print(f"Predators left: {len(detectors_history[t])}")
    print(f"Venomous Prey left: {nv_history[t]}")
    print(f"Mimic Prey left: {len(signals_history[t]) - nv_history[t]}")

    
    d, s, r, v, nv = update(
        detectors_history[t], 
        signals_history[t], 
        risk_tols_history[t], 
        venom_levels_history[t], 
        nv_history[t], 
        handling_time = ht,
        attack_freq=af,
        predator_conversion_ratio=predator_conversion_ratio,
        prey_conversion_ratio=prey_conversion_ratio,
        mutation_rate=mutation_rate, 
        phenotype_type='vector')
    
    detectors_history.append(d)
    signals_history.append(s)
    risk_tols_history.append(r)
    venom_levels_history.append(v)
    nv_history.append(nv)


 Gen 0
Predators left: 100
Venomous Prey left: 200
Mimic Prey left: 200

 Gen 1
Predators left: 315
Venomous Prey left: 434
Mimic Prey left: 435

 Gen 2
Predators left: 336
Venomous Prey left: 453
Mimic Prey left: 445

 Gen 3
Predators left: 336
Venomous Prey left: 424
Mimic Prey left: 442

 Gen 4
Predators left: 341
Venomous Prey left: 415
Mimic Prey left: 429

 Gen 5
Predators left: 381
Venomous Prey left: 444
Mimic Prey left: 414

 Gen 6
Predators left: 352
Venomous Prey left: 450
Mimic Prey left: 409

 Gen 7
Predators left: 382
Venomous Prey left: 429
Mimic Prey left: 390

 Gen 8
Predators left: 313
Venomous Prey left: 480
Mimic Prey left: 387

 Gen 9
Predators left: 352
Venomous Prey left: 495
Mimic Prey left: 409

 Gen 10
Predators left: 347
Venomous Prey left: 465
Mimic Prey left: 399

 Gen 11
Predators left: 374
Venomous Prey left: 493
Mimic Prey left: 435

 Gen 12
Predators left: 357
Venomous Prey left: 454
Mimic Prey left: 365

 Gen 13
Predators left: 341
Venomous Prey left:

In [295]:
steps_per_frame = 1
fig = plt.figure(figsize=(4,4))
ax = plt.axes(xlim=(-5, 5), ylim=(-5, 5))

def animate(i):
    # Remove old points
    for artist in ax.get_lines()+ax.collections:  
        artist.remove()

    t = i * steps_per_frame
    num_venomous_prey = nv_history[t]
    pts = []
    ax.scatter(detectors_history[t][ :, 0], detectors_history[t][ :, 1], color='red', label='predators detectors', s=0.2)
    ax.scatter(signals_history[t][ :num_venomous_prey, 0], signals_history[t][ :num_venomous_prey, 1], color='orange', label='venomous signals', s=0.2, alpha=0.5)
    ax.scatter(signals_history[t][ num_venomous_prey:, 0], signals_history[t][ num_venomous_prey:, 1], color='green', label='mimic signals', s=0.2, alpha=0.5)
    ax.scatter(0, 0, color='k', label='origin')

    return ax.collections

anim = animation.FuncAnimation(fig, animate, frames=len(detectors_history)//steps_per_frame, interval=50, blit=True)
plt.close()
anim.save('test.gif')
HTML(anim.to_jshtml())